In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import config

## Scrape USA Today to get salaries for MLB players

In [2]:
url = "https://www.usatoday.com/sports/mlb/salaries/"
#use pandas to scrape tables into list of data frames
tables = pd.read_html(url)

In [3]:
#select the first table
salaries = tables[0]
salaries.head(3)

,rank,Name,Team,POS,Salary,Years,Total Value,Avg Annual
0,--,Max Scherzer,WSH,SP,"$42,142,857",7 (2015-21),"$210,000,000","$30,000,000"
1,--,Stephen Strasburg,WSH,SP,"$36,428,571",7 (2017-23),"$175,000,000","$25,000,000"
2,--,Mike Trout,LAA,CF,"$34,083,333",6 (2015-20),"$144,500,000","$24,083,333"


## Scrape FanGraphs to get starting lineups for each team

In [4]:
#create list of all MLB team names
teams =['braves', 'marlins','mets','phillies', 'nationals', 'cubs', 'reds', 'brewers', 'pirates', 'cardinals','diamondbacks',
        'rockies', 'dodgers','padres','giants','orioles','redsox','yankees','rays','bluejays','whitesox','indians','tigers',
        'royals','twins','astros','angels','athletics', 'mariners','rangers']

In [5]:
#Create empy lists and dataframes to store data for the scrape
ndf=pd.DataFrame()
url_list = []

#loop to create list of url's for each team's lineup
for i in teams:
    url = "https://www.fangraphs.com/teams/{}/depth-chart".format(i)
    url_list.append(url)

#loop through each team's url to scrape their starting lineup
for x in url_list:
    res = requests.get(x)
    soup = BeautifulSoup(res.content, 'lxml')
    table = soup.find_all('div', {'class':'team-stats-table'})
    df = pd.read_html(str(table))

    lineup = []

    for frame in df:
        lineup.append(frame.loc[0])

    #temp df to hold single team lineup    
    tdf= pd.DataFrame(lineup)
    #add positions
    tdf['position'] = ['C','1B','2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL', 'SP', 'RP', 'AP']

    #add team names
    tn = x.split('/')
    tdf['team_name'] = tn[4]
    tdf = tdf.reset_index()
    tdf = tdf.set_index('position')
    
    #drop the old index and non-useful positions
    tdf = tdf.drop('index', axis=1)
    tdf = tdf.drop(['ALL', 'AP'])

    #add results to result df
    ndf = pd.concat([ndf,tdf])
    #clear temp df
    tdf= pd.DataFrame()


In [6]:
ndf = ndf.reset_index()
ndf['table_index'] = np.arange(len(ndf))
ndf.head(3)


,position,Name,PA,AVG,OBP,SLG,wOBA,Bat,Fld,BsR,...,IP,K/9,BB/9,HR/9,BABIP,LOB%,ERA,FIP,team_name,table_index
0,C,Brian McCann,32.0,0.242,0.322,0.408,0.312,-0.5,-0.1,-0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,braves,0
1,1B,Freddie Freeman,83.0,0.298,0.390,0.548,0.386,4.0,0.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,braves,1
2,2B,Ozzie Albies,80.0,0.279,0.333,0.469,0.335,0.3,0.7,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,braves,2


## Join the salary and lineup dataframes

In [7]:
df_merge = pd.merge(ndf, salaries, on='Name', how='left')

In [38]:
#drop unwanted columns
df_merge = df_merge.drop('rank', 1, errors = 'ignore')
df_merge = df_merge.drop('index', 1, errors = 'ignore')
df_merge = df_merge.drop('table_index', 1, errors = 'ignore')
df_merge = df_merge.drop('POS', 1, errors = 'ignore')
df_merge = df_merge.drop('Team', 1, errors = 'ignore')
df = df_merge.rename(columns={"LOB%": "LOB_percent"})
df.count()

position       330
Name           330
PA             270
AVG            270
OBP            270
SLG            270
wOBA           270
Bat            270
Fld            270
BsR            270
WAR            330
IP              60
K/9             60
BB/9            60
HR/9            60
BABIP           60
LOB_percent     60
ERA             60
FIP             60
team_name      330
Salary         270
Years          270
Total Value    270
Avg Annual     270
dtype: int64

In [39]:
#pitchers table
pitchers = df.dropna(subset=['IP', 'ERA'])
pitchers = pitchers.drop(pitchers.iloc[:, 2:10], axis = 1) 
pitchers.count()

position       60
Name           60
WAR            60
IP             60
K/9            60
BB/9           60
HR/9           60
BABIP          60
LOB_percent    60
ERA            60
FIP            60
team_name      60
Salary         56
Years          56
Total Value    56
Avg Annual     56
dtype: int64

In [40]:
#Fielders table
fielder = df.dropna(subset=['AVG', 'OBP'])
fielder = fielder.drop(fielder.iloc[:, 11:19], axis = 1) 
fielder.count()

position       270
Name           270
PA             270
AVG            270
OBP            270
SLG            270
wOBA           270
Bat            270
Fld            270
BsR            270
WAR            270
team_name      270
Salary         214
Years          214
Total Value    214
Avg Annual     214
dtype: int64

## Load into Database

In [12]:
connection_string = "{}:{}@localhost:5432/mlb_db".format(config.username, config.password)
engine = create_engine(f'postgresql://{connection_string}')

In [42]:
df.to_sql(name='starting_rosters', con=engine, if_exists='replace', index=True)

In [41]:
pitchers.to_sql(name='pitchers', con=engine, if_exists='replace', index=False)
fielder.to_sql(name='fielders', con=engine, if_exists='replace', index=False)